In [ ]:
import numpy as np
import torch
from binary_btsp_network_torch import BinaryBTSPNetworkTorch


# -------------------------
# 参数设置
# -------------------------
n_input = 20000
n_memory = 30000
fw = 0.6
fq = 0.0025
fp = 0.005
threshold = 3
n_patterns = 1500        # 训练的模式数量
mask_ratios = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7,0.9]  # 掩码比例
n_repeats = 10          # 重复次数取平均

# -------------------------
# 初始化网络
# -------------------------
net = BinaryBTSPNetworkTorch(
    n_input=n_input,
    n_memory=n_memory,
    fw=fw,
    fq=fq,
    fp=fp,
    threshold=threshold,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# -------------------------
# 生成训练模式
# -------------------------
patterns = net.make_sparse_patterns(n_patterns)

# -------------------------
# 训练网络
# -------------------------
net.train(patterns)
    


In [ ]:
# ------------------------- # 测试寻回程度 # ------------------------- 
rel_diss = net.compute_relative_dissimilarity(
     patterns=patterns, 
     mask_ratios=mask_ratios,
     n_repeats=n_repeats, ) 
# ------------------------- # 打印结果 # ------------------------- 
print("掩码比例:", mask_ratios) 
print("平均相对不相似度:", rel_diss)

In [ ]:
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# -------------------------
# 绘图
# -------------------------
plt.figure(figsize=(6, 4))
plt.plot(mask_ratios, rel_diss, marker="o", linestyle="-", color="tab:blue", label="平均相对不相似度")

plt.xlabel("掩码比例", fontsize=12)
plt.ylabel("平均相对不相似度", fontsize=12)
plt.title("掩码比例 vs 平均相对不相似度", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()




In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from tqdm import tqdm
import torch
from binary_btsp_network_torch import BinaryBTSPNetworkTorch


# ---------------- 参数 ----------------
n_input = 200       # 神经元数量
n_memory = 300     # CA1 神经元数量
fw = 0.6             # 连接稀疏度
fq = 0.005            # BTSP 更新概率
fp = 0.005            # 输入模式稀疏度
n_repeats = 10        # 相对不相似度重复次数

device = "cuda" if torch.cuda.is_available() else "cpu"

# 掩码比例范围
mask_range = np.linspace(0.0, 0.9, 10)

# 模式数量范围
pattern_range = np.linspace(0.1 * n_memory, 0.9 * n_memory, 10).astype(int)


# 结果矩阵
results = np.zeros((len(pattern_range), len(mask_range)))

# ---------------- 循环不同模式数量 ----------------
for i, n_patterns in enumerate(tqdm(pattern_range, desc="模式数量")):
    # 初始化网络
    net = BinaryBTSPNetworkTorch(
        n_input=n_input,
        n_memory=n_memory,
        fq=fq,
        fp=fp,
        fw=fw,
        threshold=3,
        device=device
    )

    # 生成随机稀疏输入模式
    patterns = net.make_sparse_patterns(n_patterns)

    # 训练网络
    net.train(patterns)

    # ---------------- 计算不同掩码比例的相对不相似度 ----------------
    rel_diss = net.compute_relative_dissimilarity(
        patterns=patterns,
        mask_ratios=mask_range,
        n_repeats=n_repeats
    )

    results[i, :] = rel_diss

# ---------------- 绘制三维曲面图 ----------------
X, Y = np.meshgrid(mask_range, pattern_range)
Z = results

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="3d")
surf = ax.plot_surface(X, Y, Z, cmap="viridis")

ax.set_xlabel("掩码比例")
ax.set_ylabel("记忆模式数量")
ax.set_zlabel("相对不相似度")
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()


